# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm#MID), thank goodness we can search for these things.

## Preparation: Knowing your tags

These questions are the same for every data set, and might not work exactly for yours.

**Search for every operator with 'dirt' in their name, including abandoned mines.**

### What is the tag and class name for every row of data?

In [1]:
# tr

### What is the tag and class name for every mine operator's name?

In [2]:
# td[2]

### What is the tag and class name for every mine's name?

In [3]:
# td[3]

## Being lazy

If you only needed these results, what would you do instead of scraping them?

In [4]:
# copy-paste into Excel

## Setup: Import what you'll need to scrape the page

Use `requests`, not `urllib`.

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Try to scrape the page

To test if you requested the page correctly, save the BeautifulSoup document as `doc` and run the code `doc.find_all('tr')[-1].text` to get the text of the last `<tr>` element.

- If the result starts with **Total Number of Mines Found**, you were successful.

In [6]:
url = 'https://arlweb.msha.gov/drs/ASP/OprNameStatesearch.asp'

data = {
    'OperSearch': 'dirt',
    'MineName': '',
    'StateSearch': 'None',
    'CM': 'All',
    'x': '47',
    'y': '4',
    'MC': 'Opersearch'
}

response = requests.post(url, data=data)
doc = BeautifulSoup(response.text, 'html.parser')

In [7]:
doc.find_all('tr')[-1]

<tr>
<td bgcolor="#000000" colspan="8"><font style="FONT-SIZE:.75em; color:#FFF"><b>Total Number of Mines Found:</b>  <b>129</b></font></td></tr>

## Actually scraping

### Hopefully you know that each `tr` is supposed to be your data. What is the index of the first row element that is actually a result?

`.text` will help you here.

In [8]:
doc.find_all('tr')[7]

<tr>
<td align="center">
<form action="/drs/ASP/BasicMineInfostatecounty.asp" method="post" name="search">
<input name="MineId" type="hidden" value="3503598"/><font style="FONT-SIZE:.75em;">3503598</font>
</form></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --><b>OR</b><!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --> Newberg Rock &amp; Dirt<!-- /DNT -->  </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT -->Newberg Rock &amp; Dirt<!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->Surface             <!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->M<!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;">Active  </font></td>
<td><font style="FONT-SIZE:.75em;">Crushed, Broken Stone NEC  </font></td>
<th bgcolor="#000000"><input alt="More Information" border="0" name="submit" src="/drs/images/moreinfo.jpg" type="image"/></th></tr>

### Loop through each operator result, printing its name

Use LIST SLICING to skip the non-data row(s).

In [9]:
for row in doc.find_all('tr')[7:-1]:
    print(row.find_all('td')[2].text.strip())

Newberg Rock & Dirt
Allied Dirt Moving Company
AM Dirtworks & Aggregate Sales
Atlas-Dirty Devil Mining
Atlas-Dirty Devil Mining
Babe's Dirt Work
Bar-Lin Dirt Company
Barber'S Dirt Pit
Bender Sand & Dirt
BERT'S DIRT
Big D Dirt Service Inc
Big Red Dirt Farm LLC
Big River Dirt Pit
Bob Harris Dirt Contracting
Bohannon Sand & Dirt
Bratcher'S Sand & Dirt
Brewer Dirt Works
Buck'S Dirt Pit
C & G Dirt Hauling
C N C Dirt Movers, Inc.
Cambridge Dirt Sand and Gravel LLC
Central Iowa Dirt & Demo LLC
Crowes Trucking & Dirt Pit Services
D & H Dirt
Diez Dirt & Sand Hauling Inc
Dirt Cheap
Dirt Company
Dirt Company
Dirt Company
Dirt Con
Dirt Diggers Inc
Dirt Doctor Inc
Dirt Inc
Dirt Pit
Dirt Works
Dirtco Inc
Dirtman Trucking
DIRTWORKS, INC.
Dirtworks, Inc.
Dirty Coal
Dorchester Dirt Company Inc
Douglas Dirt Sand & Gravel Company
Ell Dirt Works LLC.
Floyd Smith Dirt Pit
Gary Kelm Dirt Service
Gerald Fenger/Rock & Dirt Const
Gerald Illies Gravel & Dirt Company
Guidry Sand & Dirt Pit Inc
Harris Dirt Compan

### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [10]:
for row in doc.find_all('tr')[7:-1]:
    print(row.find_all('td')[0].text.strip())

3503598
0502030
4801789
4201449
4201450
1002257
1601167
4103265
1401575
1700776
1601251
0301963
1601082
3401751
1600916
3401211
0301267
1600956
2200033
0504953
3401929
1302445
1601106
3400915
1600983
4503200
3401266
3401468
5001797
4608254
1510279
2103723
0100776
4104016
4104757
0301729
0404851
2200734
5002028
1513393
3800602
3101630
3200860
3401762
2103517
2402626
2103181
1601124
1601150
4703427
0801306
2501216
3200965
2901371
2901544
2901709
4102355
4102420
4102869
4102951
4102958
4104876
3003502
4103258
3901432
2103556
1601250
1600908
1600953
4104185
2901536
3609624
3800709
3609931
1601257
0801275
1601379
1601380
1601381
1601134
1601165
3901042
1601194
4104054
4801674
2402474
1600920
4102955
4103107
1512530
1515619
1518318
4405366
4407196
1519685
1519799
4407003
2602570
2402503
4407296
1519273
4407270
4102682
0801259
0203332
0302015
2901986
1601127
4105017
1600986
4103324
4202013
0801371
2402115
4300748
4300768
4300776
0103209
1601159
2302283
4102586
4104475
3800617
1601234
4104648


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

In [11]:
mines = []

for mine in doc.find_all('tr')[7:-1]:
    mines.append({
        'Operator ID': mine.find_all('td')[0].text.strip(),
        'Operator name': mine.find_all('td')[2].text.strip(),
        'Mine name': mine.find_all('td')[3].text.strip(),
        'State': mine.find_all('td')[1].text.strip(),
        'Mine type': mine.find_all('td')[4].text.strip(),
        'Coal or metal': mine.find_all('td')[5].text.strip(),
        'Status': mine.find_all('td')[6].text.strip(),
        'Commodity': mine.find_all('td')[7].text.strip(),
    })

### Save that to a CSV

In [12]:
df = pd.DataFrame(mines)
df.to_csv('mines_list.csv', index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [13]:
pd.read_csv('mines_list.csv')

,Coal or metal,Commodity,Mine name,Mine type,Operator ID,Operator name,State,Status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,3503598,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,Allied Dirt Moving Co Pit & Plant,Surface,502030,Allied Dirt Moving Company,CO,Abandoned
2,M,Construction Sand and Gravel,AM Dirtworks & Aggregate Sales,Surface,4801789,AM Dirtworks & Aggregate Sales,ND,Intermittent
3,C,Coal (Bituminous),Unit Train Loading Facility,Facility,4201449,Atlas-Dirty Devil Mining,UT,Abandoned
4,C,Coal (Bituminous),Blackie Surface Mine & Prep Plant,Surface,4201450,Atlas-Dirty Devil Mining,UT,Abandoned
5,M,Construction Sand and Gravel,"Hitt Pit, Inc.",Surface,1002257,Babe's Dirt Work,ID,Abandoned
6,M,Construction Sand and Gravel,Bar-Lin Dirt Pit,Surface,1601167,Bar-Lin Dirt Company,LA,Abandoned
7,M,Construction Sand and Gravel,Barber'S Dirt Pit,Surface,4103265,Barber'S Dirt Pit,TX,Abandoned
8,M,Construction Sand and Gravel,BENDER SAND & DIRT,Surface,1401575,Bender Sand & Dirt,KS,Abandoned
9,M,Construction Sand and Gravel,BERT'S DIRT,Surface,1700776,BERT'S DIRT,ME,Abandoned
